In [109]:
import pandas as pd
import numpy as np
import json
import requests
import os
from googleapiclient.discovery import build

In [110]:
# load the playlists data from the csv
playlists = pd.read_csv('playlist-clean.csv')
playlists.head()

,Unnamed: 0,url,tweet_created_at,twitter_user_id,list_id
0,0,https://youtube.com/playlist?list=PL6BT4U83o4J...,2021-06-04 12:58:58 UTC,1.400000e+18,PL6BT4U83o4Je3TDUWwkPYazR7FY9CtySw
1,1,https://www.youtube.com/playlist?list=PLDVYaMO...,2018-12-14 05:27:20 UTC,1.070000e+18,PLDVYaMOAufIDGuZ2p50Q9EO5f1gNUcUIZ
2,2,https://www.youtube.com/playlist?list=PLEeIGZu...,2022-10-17 03:51:24 UTC,1.580000e+18,PLEeIGZuBXtsuRJRtQH2P8xUGi6AR_MN4w
3,3,https://www.youtube.com/playlist?list=PLhOQAKe...,2019-04-01 02:30:05 UTC,1.110000e+18,PLhOQAKeRPuGkhS5mSDgdXmFV-h9mRMf24
4,4,https://www.youtube.com/playlist?list=PLEeIGZu...,2022-10-20 04:51:09 UTC,1.580000e+18,PLEeIGZuBXtstpI4hORi6_exCErOMyl-9_


In [111]:
# Droping the extra column and renaming the twitter user id and created at column
playlists.drop(['Unnamed: 0'], axis=1, inplace=True)
playlists.rename(columns = {'id':'twitter_user_id', 'created_at': 'tweet_created_at'}, inplace = True)
playlists.head()

,url,tweet_created_at,twitter_user_id,list_id
0,https://youtube.com/playlist?list=PL6BT4U83o4J...,2021-06-04 12:58:58 UTC,1.400000e+18,PL6BT4U83o4Je3TDUWwkPYazR7FY9CtySw
1,https://www.youtube.com/playlist?list=PLDVYaMO...,2018-12-14 05:27:20 UTC,1.070000e+18,PLDVYaMOAufIDGuZ2p50Q9EO5f1gNUcUIZ
2,https://www.youtube.com/playlist?list=PLEeIGZu...,2022-10-17 03:51:24 UTC,1.580000e+18,PLEeIGZuBXtsuRJRtQH2P8xUGi6AR_MN4w
3,https://www.youtube.com/playlist?list=PLhOQAKe...,2019-04-01 02:30:05 UTC,1.110000e+18,PLhOQAKeRPuGkhS5mSDgdXmFV-h9mRMf24
4,https://www.youtube.com/playlist?list=PLEeIGZu...,2022-10-20 04:51:09 UTC,1.580000e+18,PLEeIGZuBXtstpI4hORi6_exCErOMyl-9_


In [112]:
playlists.isna().sum()

url                 0
tweet_created_at    0
twitter_user_id     0
list_id             0
dtype: int64

In [113]:
playlists.isnull().sum()

url                 0
tweet_created_at    0
twitter_user_id     0
list_id             0
dtype: int64

In [114]:
# the data is clean
# urls_with_empty_id = playlists[playlists['list_id'].isnull() | playlists['list_id'].eq('')]['url']
# print(urls_with_empty_id)

In [115]:
# playlists.dropna(subset=['list_id'], inplace=True)
# playlists

In [116]:
playlists['list_id'][0:3]

0    PL6BT4U83o4Je3TDUWwkPYazR7FY9CtySw
1    PLDVYaMOAufIDGuZ2p50Q9EO5f1gNUcUIZ
2    PLEeIGZuBXtsuRJRtQH2P8xUGi6AR_MN4w
Name: list_id, dtype: object

In [117]:
error_ids = set()
with open('error_ids.txt') as f:
    for line in f:
        error_ids.add(line.strip())
# print(error_ids)

In [118]:
# Creating subsets of data - list_id
ids = list(set(playlists['list_id']))

p1 = ids[0:1]
p3 = ids[0:3]
p10 = ids[0:10]
p100 = ids[0:100]

len(ids)

2481

In [119]:
final_ids = []
for id_ in ids:
    if id_ not in error_ids:
        final_ids.append(id_)

len(final_ids)

2067

In [120]:
final_ids.remove('RDAMVMBJWOlYsceGc')

In [121]:
final_ids.remove('PLENzd1lcrhLO5fHjhb1Ajs2l678nwVNHq')

In [122]:
final_ids.remove('RDpnMtmFzMLP4')

In [123]:
final_ids.remove('RDvOnRcfhqh3s') # unviewable

In [124]:
final_ids.remove('RDCLAK5uy_lhKsFGmKkGIdQFU4VmpyHI1sgwNu4n-KI')

In [125]:
final_ids.remove('RDCLAK5uy_lS6GrniXk1dG8pfexKxJPdN7WSYK61l5w')

In [126]:
final_ids.remove('RDCLAK5uy_lOAAW-PX5XUed76iQefCxkOXd6m6ZvyiM')

In [127]:
final_ids.remove('RDCLAK5uy_nMDciv8eorZEJB5uuWvd6twAxQ13LIEvg')

In [128]:
final_ids.remove('PLHg1sVThC-gOV_2LmFfK3J1isqPYs2H70')

In [129]:
final_ids.remove('RDOnkwhU6_Dxg')

In [130]:
final_ids.remove('RDCLAK5uy_n0vqPVYwwLGVv8XMpjj7IovO50hqegreo')

In [131]:
final_ids.remove('RDCLAK5uy_nug06gQBs3XidJ9RyjdniwqiT3yTKLHTI')

In [138]:
final_ids.remove('RDCLAK5uy_mfut9V_o1n9nVG_m5yZ3ztCif29AHUffI')

In [139]:
len(final_ids)

2054

In [140]:
d1 = final_ids[0:1000]
d2 = final_ids[1000:len(final_ids)]

In [141]:
# yt data api_key
with open('api_key.txt', 'r') as f:
    api_key = f.read()

In [150]:
data_file = 'playlist_data.json'

if os.path.exists(data_file):
    with open(data_file, 'r') as file:
        all_data = json.load(file)
    
print(len(all_data))

2058


In [144]:
def fetch_playlist_videos_data(playlist_ids, api_key):
    ''' Function that takes a list of playlist_ids as input and an api_key and saves all the data in a 
    playlist_data.json file. It uses a try except block for error handling and also checks for a 
    id which is already fetched making less api calls '''
    data_file = 'playlist_data.json'
    all_data = {}

    # Load existing data from the JSON file
    if os.path.exists(data_file):
        print(1)
        with open(data_file, 'r') as file:
            all_data = json.load(file)

    for playlist_id in playlist_ids:
        # Check if playlist data already exists in the JSON
        if (playlist_id in all_data) and all_data[playlist_id] != None:
            print(f"Playlist ID {playlist_id} data already fetched. Skipping...")
            continue

        playlist_data = []
        next_page_token = None

        try:
            while True:
                print(playlist_id)
                url = f"https://www.googleapis.com/youtube/v3/playlistItems?part=snippet%2CcontentDetails%2Cid&maxResults=50&playlistId={playlist_id}&key={api_key}"
                if next_page_token:
                    url += f"&pageToken={next_page_token}"

                response = requests.get(url)
                data = json.loads(response.text)

                if 'error' in data and data['error']['message'] == 'This playlist type is unviewable.':
                    print(f"Playlist ID {playlist_id} is unviewable. Storing null value...")
                    all_data[playlist_id] = None
                    break

                for item in data['items']:
                    video_data = {}
                    video_id = item['contentDetails']['videoId']
                    video_data['snippet'] = item['snippet']
                    # Add any additional data you want to include for each video

                    playlist_data.append({video_id: video_data})

                next_page_token = data.get('nextPageToken')
                if not next_page_token:
                    break

        except Exception as e:
            print(f"Error fetching data for playlist ID {playlist_id}: {str(e)}")
            all_data[playlist_id] = None
            continue

        all_data[playlist_id] = playlist_data

        # Save the updated data to the JSON file
        with open(data_file, 'w') as file:
            json.dump(all_data, file, indent=4)

    return all_data


In [145]:
last_1000 = fetch_playlist_videos_data(d2, api_key=api_key)

1
Playlist ID PL-ujIE4tLGbGengnag-PscbYjVn77pTXg data already fetched. Skipping...
Playlist ID PL-ujIE4tLGbFeN97PCOgwQC6xh0tqNdgN data already fetched. Skipping...
Playlist ID PLZxP0elzkoDNimHL4dZJHSQ5ZSm3LA5O0 data already fetched. Skipping...
Playlist ID PLoAoFXEnRkKFHZIbLIET0jy1jvFmhSsOR data already fetched. Skipping...
Playlist ID OLAK5uy_lpHUSMEBgUPKk3Viw1biU4Yy1P6vTsaTY data already fetched. Skipping...
Playlist ID PLvsYXqtYjMYdnFAD77W56tkWtTQUwIFRA data already fetched. Skipping...
Playlist ID PLdbXyyVfVp-7hBkVKQ3teqZDpkN3MQ4yz data already fetched. Skipping...
Playlist ID OLAK5uy_kg1lEtEBuMYOJsnoiw_kKZdyByM9Ip098 data already fetched. Skipping...
Playlist ID PLyBTDgLIIenZQ9Avxx14YrxabMO0q3iGr data already fetched. Skipping...
Playlist ID PL7pGJQV-jlzD17YNNbt103xp0PkkUCoPU data already fetched. Skipping...
Playlist ID PLADCA51117140A145 data already fetched. Skipping...
Playlist ID PLkcywTrP0B0lqSjYVbfwjAxke29RXrYIU data already fetched. Skipping...
Playlist ID PLkETZOTiwuvixzo

In [146]:
print(1)

1


In [147]:
# for the 1st 1000 playlist links
first_1000 = fetch_playlist_videos_data(d1, api_key=api_key)

1
PL6Trj3kSb1FmjykRfr6GZYtqGDb4RAjWb
OLAK5uy_l7_wU9NJh9m5pW8nWXaCbuEnZfhHPFGsA
OLAK5uy_mfWQa8MsoE0oN_13o4RLa4gLFeuNxRVpg
Playlist ID PL_rAdiZmN7vGl-9E5Dil0okNK9KtUGIrI data already fetched. Skipping...
OLAK5uy_meD0h9BUd3BOH1FV1ygvTFEYRbKw0ewp4
PLP32wGpgzmIlDFQuxl1qeH33hJJSq8wfe
PLP32wGpgzmIlDFQuxl1qeH33hJJSq8wfe
Playlist ID PLvthR4YhEPYAGYR5cPNKuXJLrbX-rl6jP data already fetched. Skipping...
PLzWfZcUCF2DrlhVOLvhOaoGcXXYPjJBaw
PLNc26k4M0SlNi4Eu-g2T7N1UEs2XcuiRr
Playlist ID PLhxITUni9OWes4XXblfTGR0RcbKgoGGKB data already fetched. Skipping...
Playlist ID OLAK5uy_l9hBJ3rHyZPC9YHBtPrnpIL6sejNLjgKs data already fetched. Skipping...
PLxq_lXOUlvQD6xBZm3sS0qQ7Bjj_o4-qx
Playlist ID PLYvhPWR_XYJnPvrhXE4RYvwZhV26nYTIp data already fetched. Skipping...
PLTzKiTp7N5nGgvzhsL6tevuMp4ThjNXLl
PLXuY2YL-v4n_PLD6ybk_9u0jhoUZvWwVv
PLXuY2YL-v4n_PLD6ybk_9u0jhoUZvWwVv
PLbUh9sjiMWsVIAgmzLvZLA2ZkV-l_prQ4
Playlist ID PL2hoGhz2jBSr0F3k9hIYqsllrqKlMLn1V data already fetched. Skipping...
OLAK5uy_kGPjQ2nb7ukJdsqVBtO37

In [149]:
print("hi")

hi


In [60]:
# To fetch the data from teh aove 32 videos we need an authorization token
headers = {
    'Authorization': 'YOUR_AUTH_HEADER',
    'Content-Type': 'application/json'
}
# response = requests.get(url, headers=headers)
# data = json.loads(response.text)

In [62]:
# this script is written to change/replace all the empty lists [] by None

# data_file = 'playlist_data.json'
# all_data = {}

# # Load existing data from the JSON file
# if os.path.exists(data_file):
#     with open(data_file, 'r') as file:
#         all_data = json.load(file)
# print('Read File')

# for id in all_data.keys():
#     if all_data[id] == []:
#         all_data[id] = None
# print('changes made')

# with open(data_file, 'w') as file:
#     json.dump(all_data, file, indent=4)
# print('write done')